In [1]:
%pip install -q -U langchain-aws

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip show boto3 botocore

Name: boto3
Version: 1.35.24
Summary: The AWS SDK for Python
Home-page: https://github.com/boto/boto3
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /opt/conda/lib/python3.11/site-packages
Requires: botocore, jmespath, s3transfer
Required-by: amazon-q-developer-jupyterlab-ext, amazon-sagemaker-sql-editor, amazon-sagemaker-sql-execution, amazon_sagemaker_jupyter_ai_q_developer, autogluon.common, autogluon.core, autogluon.multimodal, aws-glue-sessions, langchain-aws, llama-index-llms-bedrock, PyAthena, redshift_connector, sagemaker, sagemaker-mlflow, sagemaker-studio-analytics-extension, sagemaker-studio-sparkmagic-lib
---
Name: botocore
Version: 1.35.24
Summary: Low-level, data-driven core of boto 3.
Home-page: https://github.com/boto/botocore
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /opt/conda/lib/python3.11/site-packages
Requires: jmespath, python-dateutil, urllib3
Required-by: aiobotocore, amazon-q-developer-ju

In [5]:
# Refresh all variables, aliases and directory history. from store
%store -r

In [3]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [4]:
import boto3
import json

In [5]:
client = boto3.client("bedrock-runtime")
agent_client = boto3.client("bedrock-agent-runtime")

In [6]:
def retrieve(query, kbId, numberOfResults=5):
    return agent_client.retrieve(
        retrievalQuery= {'text': query},
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # optional
            }
        }
    )

In [8]:
query = "What is Amazon doing in the field of generative AI?"
kb_id = "RD3KHHLG96"
response = retrieve(query, kb_id, 5)
retrievalResults = response['retrievalResults']


In [9]:
import pprint
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(retrievalResults)

[ { 'content': { 'text': 'The customer reaction to what we’ve shared thus far '
                         'about Kuiper has been very positive, and we believe '
                         'Kuiper represents a very large potential opportunity '
                         'for Amazon. It also shares several similarities to '
                         'AWS in that it’s capital intensive at the start, but '
                         'has a large prospective consumer, enterprise, and '
                         'government customer base, significant revenue and '
                         'operating profit potential, and relatively few '
                         'companies with the technical and inventive aptitude, '
                         'as well as the investment hypothesis to go after '
                         'it.   One final investment area that I’ll mention, '
                         'that’s core to setting Amazon up to invent in every '
                         'area of our business for 

In [10]:
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts
contexts = get_contexts(retrievalResults)
pp.pprint(contexts)

[ 'The customer reaction to what we’ve shared thus far about Kuiper has been '
  'very positive, and we believe Kuiper represents a very large potential '
  'opportunity for Amazon. It also shares several similarities to AWS in that '
  'it’s capital intensive at the start, but has a large prospective consumer, '
  'enterprise, and government customer base, significant revenue and operating '
  'profit potential, and relatively few companies with the technical and '
  'inventive aptitude, as well as the investment hypothesis to go after it.   '
  'One final investment area that I’ll mention, that’s core to setting Amazon '
  'up to invent in every area of our business for many decades to come, and '
  'where we’re investing heavily is Large Language Models (“LLMs”) and '
  'Generative AI. Machine learning has been a technology with high promise for '
  'several decades, but it’s only been the last five to ten years that it’s '
  'started to be used more pervasively by companies. This s

In [14]:
prompt_data = f"""
Human: You are a financial advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{contexts}
</context>

<question>
{query}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

In [16]:
resp = client.invoke_model(
    modelId='mistral.mistral-7b-instruct-v0:2',
    contentType='application/json',
    accept='application/json',
    body=json.dumps({
        "prompt": prompt_data,
        "max_tokens":512,
        "temperature":0.5,
        "top_k":50,
        "top_p":0.9
    })
)

In [17]:
body = json.loads(resp.get("body").read())

In [22]:
print(body['outputs'][0]['text'])

 Amazon is investing substantially in Large Language Models (LLMs) and Generative AI. They have been working on their own LLMs for some time, which they believe will transform and improve virtually every customer experience. Amazon is also democratizing this technology by offering the most price-performant machine learning chips (Trainium and Inferentia) for small and large companies to train and run their LLMs in production. They are delivering applications like AWS's CodeWhisperer, which generates code suggestions in real time, revolutionizing developer productivity. Amazon's investment in LLMs and Generative AI is significant and they believe it will be transformative for their business and their customers.
